In [ ]:
# Importamos las librerías necesarias
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException
from selenium.webdriver.support.ui import WebDriverWait
# import xlrd
from src.soporte_scrapeo_general import get_competiciones, cambio_pestaña, resultados_disciplina, buscador_elementos, guardado_info, competiciones_año, obtencion_año, creacion_dictios_guardado
# from src.soporte_scrapeo_salto import extraccion_resultados_jinetes_caballos

## Concursos nacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_doma_nac, dictio_pruebas_doma_nac, dictio_jinetes_doma_nac, dictio_caballos_doma_nac = creacion_dictios_guardado()
urls_resultados_doma_nac = []

driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, disciplina = "completo")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2017:

    # vamos a todos los concursos del año en el que estemos
    competiciones_año(driver)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,21)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:

        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(1, driver)
                time.sleep(2)
                contenido_general =  buscador_elementos(driver, "/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.back()
                    time.sleep(1)

                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    print(contenido_general)
                    
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_concursos_doma_nac, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(2, driver)
                    time.sleep(1)

                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_concursos_doma_nac, dictio_pruebas_doma_nac, urls_resultados_doma_nac, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_concursos_doma_nac, dictio_pruebas_doma_nac, urls_resultados_doma_nac)

                    driver.close()
                    cambio_pestaña(1, driver)
                    driver.back()
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()        
    año = obtencion_año(driver)

## Concursos internacionales

In [ ]:
# url inicial
url = "https://rfhe.com/competiciones/"

dictio_concursos_doma_int, dictio_pruebas_doma_int, dictio_jinetes_doma_int, dictio_caballos_doma_int = creacion_dictios_guardado()
urls_resultados_doma_int = []

# inicializamos el driver y lo abrimos
driver = get_competiciones(url)
time.sleep(2)

# Seleccionamos el ambito de los concursos y la disciplina, en este caso nacional y completo
ambito_buscado, disciplina_buscada = resultados_disciplina(driver, ambito = "internacional", disciplina = "completo")
cambio_pestaña(1,driver)
time.sleep(2)

año = obtencion_año(driver)

while año >= 2024:

    # Buscamos los concursos del año que nos aparece
    competiciones_año(driver)

    time.sleep(3)

    maximo_n_concursos = int(buscador_elementos(driver, "/html/body/form/div/div/div/ul/li[13]/font").text.split(" ")[-1].replace("(", "").replace(")", ""))
    if año == 2025:
        rango = range(5,33)
    elif año < 2025:
        rango = range(5, maximo_n_concursos + 5) 

    for i in rango:
        
        try:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/a")
            print(concurso_bueno.text)
            time.sleep(1)
            try:
                concurso_bueno.click()
                cambio_pestaña(2, driver)
                time.sleep(2)
                contenido_general = buscador_elementos(driver, "/html/body/table/tbody/tr[2]/td/table[2]/tbody/tr[1]/td[2]/table").text.split('\n')
                                                                
                if "Resultados: Ver resultados  Ver" not in contenido_general:
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(1) 
                
                elif "Resultados: Ver resultados  Ver" in contenido_general:
                    extraccion_info_concursos(driver, diccionario_concursos=dictio_pruebas_doma_int, ambito_buscado=ambito_buscado, contenido_general=contenido_general)
                    cambio_pestaña(3,driver)
                    if i == 5 and año == 2025:
                        extraccion_info_pruebas(driver, dictio_pruebas_doma_int, dictio_pruebas_doma_int, urls_resultados_doma_int, es_primer_concurso=True)
                    else:
                        extraccion_info_pruebas(driver, dictio_pruebas_doma_int, dictio_pruebas_doma_int, urls_resultados_doma_int)
                    driver.close()
                    cambio_pestaña(2, driver)
                    driver.close()
                    cambio_pestaña(1, driver)
                    time.sleep(2)

            except NoSuchElementException:
                raise NoSuchElementException

        except NoSuchElementException:
            concurso_bueno = buscador_elementos(driver,f"/html/body/form/div/div/div/div/div[13]/table/tbody/tr[{i}]/td[4]/font").text
            print(concurso_bueno)
    
    buscador_elementos(driver,"/html/body/form/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[2]/a").click()
    time.sleep(3)        
    año = obtencion_año(driver)
driver.quit()